In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

!unzip -q '/content/drive/MyDrive/mp_dataset/new-plant-diseases-dataset.zip' -d '/content'
# drive.flush_and_unmount() 


Mounted at /content/drive
replace /content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Scab 3003.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: nA
replace /content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Scab 3003_270deg.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
data_dir = '/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Disease-Detection'
train_dir = data_dir + '/test_files'
valid_dir = data_dir + '/valid'
test_dir = '/Users/manik/OneDrive/Desktop/plant/Plant-Disease-Detection/test_files'

In [3]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input, decode_predictions
from keras.utils.np_utils import to_categorical
import numpy as np
import os, shutil

In [4]:
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [6]:
newpath = '/Users/manik/OneDrive/Desktop/plant/Plant-Disease-Detection/train' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
x = os.listdir(train_dir)

for dir_names in x:
  
  p = os.listdir(os.path.join(train_dir, dir_names))
 
  if not os.path.exists(os.path.join(newpath, dir_names)):
    os.makedirs(os.path.join(newpath, dir_names))
  cnt = 0
 
  file_names = []
  for image_names in p:
    
    if cnt==250:
      break
    file_names.append(train_dir+'/'+dir_names+'/'+image_names)
    cnt += 1
  # for f in file_names:
  #   shutil.move(f, os.path.join(newpath, dir_names))
file_names



['/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Disease-Detection/test_files/dir_2/apple_black_rot.JPG',
 '/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Disease-Detection/test_files/dir_2/Apple_ceder_apple_rust.JPG',
 '/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Disease-Detection/test_files/dir_2/apple_healthy.JPG',
 '/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Disease-Detection/test_files/dir_2/Apple_scab.JPG',
 '/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Disease-Detection/test_files/dir_2/background_without_leaves.jpg',
 '/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Disease-Detection/test_files/dir_2/blueberry_healthy.JPG',
 '/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Di

In [7]:
train_datagen = ImageDataGenerator(shear_range=0.2,zoom_range=0.2,horizontal_flip=False,vertical_flip=False
                                  ,fill_mode='nearest',width_shift_range=0.2,height_shift_range=0.2)

val_datagen = ImageDataGenerator()

train = train_datagen.flow_from_directory(directory=train_dir,batch_size=32,target_size=(224,224),
                                         color_mode='rgb',class_mode='categorical',seed=42)

valid = val_datagen.flow_from_directory(directory=valid_dir,batch_size=32,target_size=(224,224),color_mode='rgb',class_mode='categorical')

test_datagen = ImageDataGenerator()

test = test_datagen.flow_from_directory(directory=test_dir,batch_size=32,target_size=(224,224),color_mode='rgb',class_mode='categorical')


Found 84 images belonging to 2 classes.
Found 84 images belonging to 2 classes.
Found 0 images belonging to 4 classes.


In [8]:
base_model_tf=EfficientNetV2S(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=38)

In [9]:
base_model_tf.trainable=False

pt=Input(shape=(224,224,3))
func=tensorflow.cast(pt,tensorflow.float32)
x=preprocess_input(func) #This function used to zero-center each color channel wrt Imagenet dataset
model_resnet=base_model_tf(x,training=False)
model_resnet=GlobalAveragePooling2D()(model_resnet)
model_resnet=Dense(128,activation='relu')(model_resnet)
model_resnet=Dense(64,activation='relu')(model_resnet)
model_resnet=Dense(38,activation='softmax')(model_resnet)


model_main=Model(inputs=pt,outputs=model_resnet)
model_main.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.cast (TFOpLambda)        (None, 224, 224, 3)       0         
                                                                 
 efficientnetv2-s (Functiona  (None, 7, 7, 1280)       20331360  
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dense_1 (Dense)             (None, 64)                8256  

In [10]:
es=EarlyStopping(monitor='accuracy',verbose=1,patience=5,mode='auto')
mc=ModelCheckpoint(filepath='/Users/manik/OneDrive/Desktop/plant/Plant-Disease-Detection',monitor='val_accuracy',verbose=1,save_best_only=True)
lr=ReduceLROnPlateau(monitor='val_accuracy',verbose=1,patience=3,min_lr=0.001)


In [11]:
model_main.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [12]:
model_main.fit(train,validation_data=valid,epochs=30,steps_per_epoch=200,verbose=1,callbacks=[ es, lr])

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "c:\Users\manik\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\manik\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\manik\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\manik\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "C:\Users\manik\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\manik\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py"

In [13]:
model_main.save("efficientnetv2s_PLANT_DISEASE.h5")

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from PIL import Image

In [15]:
plt.figure(figsize=(10,5))
plt.plot(model_main.history.history['loss'],color='b',label='Training loss')
plt.plot(model_main.history.history['val_loss'],color='r',label='Validation loss')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.title("model loss")

KeyError: 'loss'

<Figure size 1000x500 with 0 Axes>

In [37]:
plt.figure(figsize=(10,5))
plt.plot(model_main.history.history['accuracy'],color='b',label='Training accuracy')
plt.plot(model_main.history.history['val_accuracy'],color='r',label='Validation accsuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend(['train', 'validation'], loc='lower right')
plt.title("model accuracy")

KeyError: 'accuracy'

<Figure size 1000x500 with 0 Axes>

In [16]:
loss, accuracy = model_main.evaluate(train, verbose=1)
print("Train: accuracy = %f  ;  loss = %f" % (accuracy, loss))

loss, accuracy = model_main.evaluate(valid, verbose=1)
print("Validation: accuracy = %f  ;  loss = %f" % (accuracy, loss))

loss, accuracy = model_main.evaluate(test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "c:\Users\manik\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\manik\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\manik\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\manik\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "C:\Users\manik\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\manik\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py"

In [17]:
model = load_model("/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/models/cnn/efficientnetv2s_PLANT_DISEASE.h5")

In [18]:
img = tensorflow.keras.utils.load_img(
    "/Users/manik/OneDrive/Desktop/plant/crop-recommendation-and-plant-disease-detection/Plant-Disease-Detection/test_files/dir_1/tomato_mosaic_virus.JPG",
    target_size=(224, 224, 3)
)
input_arr = tensorflow.keras.preprocessing.image.img_to_array(img)
input_arr = np.array([input_arr])
result = model.predict(input_arr)

1/1 [==============================] - 2s 2s/step


In [19]:
probability_model = tensorflow.keras.Sequential([model, 
                                         tensorflow.keras.layers.Softmax()])

In [20]:
predict = probability_model.predict(input_arr)

1/1 [==============================] - 2s 2s/step


In [21]:
predict[0]

array([0.0261475 , 0.02665516, 0.02626729, 0.02623844, 0.02633672,
       0.02627249, 0.02603434, 0.02600924, 0.0261098 , 0.0267342 ,
       0.02614021, 0.02608783, 0.02604361, 0.02649314, 0.02624973,
       0.02649402, 0.02626592, 0.0261513 , 0.02638628, 0.02671186,
       0.02616275, 0.0263364 , 0.02649173, 0.02636652, 0.02641533,
       0.02605382, 0.02627594, 0.02629775, 0.02609884, 0.02616836,
       0.02632797, 0.02602742, 0.02647477, 0.02643679, 0.02638614,
       0.0264516 , 0.02627719, 0.02712163], dtype=float32)

In [22]:
classes = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [24]:
p = np.argmax(predict[0])
print(classes[p])

Tomato___healthy
